In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] =  os.getenv("OPENAI_API_KEY")
#Langsmith
os.environ["LANGCHAIN_API_KEY"] =  os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] =  os.getenv("LANGCHAIN_PROJECT")

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(web_paths=("https://python.langchain.com/docs/introduction",),)
docs = loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splitted_text_doc = text_splitter.split_documents(docs)
splitted_text_doc

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Au

In [7]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001FF78E67C40>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001FF7B4DCB80>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [8]:
from langchain_community.vectorstores import FAISS  
faiss_vec_db = FAISS.from_documents(splitted_text_doc,embeddings)
faiss_vec_db

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
"""
Answer the following questions based on the context:
<context>
{context}
</context>

"""
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based on the context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001FF7F86CCD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001FF7F86FA30>, root_client=<openai.OpenAI object at 0x000001FF7FC35AE0>, root_async_client=<openai.AsyncOpenAI object at 0x000001FF7F86CD60>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 

In [11]:
from langchain.chains import create_retrieval_chain
faiss_retriever = faiss_vec_db.as_retriever()

retrieval_chain = create_retrieval_chain(faiss_retriever,document_chain)
retrieval_chain




RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FF7FC34BE0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following questions based on the context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
            | ChatOpenAI(clie

In [13]:
response = retrieval_chain.invoke({"input":"LangChain simplifies every stage of the LLM application lifecycle:"})
print(response['answer'])

Sure, here are the answers to your questions based on the provided context:

1. **What is LangChain?**  
   LangChain is a framework designed to develop applications that are powered by large language models (LLMs).

2. **What stages of the LLM application lifecycle does LangChain simplify?**  
   LangChain simplifies all stages of the LLM application lifecycle, including creating embeddings, developing custom LLM classes, creating tools, debugging LLM apps, and handling various data formats among others.

3. **What are some of the file types LangChain can load?**  
   LangChain can load a multitude of file types including CSVs, documents from a directory, HTML, JSON, Markdown, Microsoft Office files, PDFs, and web pages.

4. **What is LangGraph?**  
   LangGraph is an orchestration framework used to combine LangChain components into production-ready applications with features like persistence and streaming. It is also used to build stateful agents with first-class streaming and human-

In [14]:
response

{'input': 'LangChain simplifies every stage of the LLM application lifecycle:',
 'context': [Document(id='786c07c9-5fce-4bad-841f-f492665e9027', metadata={'source': 'https://python.langchain.com/docs/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:'),
  Document(id='ecac5a2d-aa73-4c26-b376-c987a4874dce', metadata={'source': 'https://python.langchain.com/docs/introduction', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='EmbeddingsHow to create a custom LLM classCustom RetrieverHow to create toolsHow to debug your LLM appsHow to load CSVsHow t